In [20]:
from scipy.stats import norm
import math

In [2]:
# First define useful functions
# probabilities a normal lies in an interval

In [3]:
# the normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = norm.cdf

In [4]:
# it's above the threshold if it's not below the threshold

In [5]:
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - norm.cdf(lo, mu, sigma)

In [6]:
# it's between if it's less than hi, but not less than lo
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return norm.cdf(hi, mu, sigma) - norm.cdf(lo, mu, sigma)

In [7]:
# it's outside if it's not between
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [8]:
#  normal bounds

In [9]:
def normal_upper_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z <= z) = probability"""
    return norm.ppf(probability, mu, sigma)

In [10]:
z = normal_upper_bound(0.975)
z

1.959963984540054

In [11]:
def normal_lower_bound(probability, mu=0, sigma=1):
    """returns the z for which P(Z >= z) = probability"""
    return norm.ppf(1 - probability, mu, sigma)

In [12]:
w = normal_lower_bound(0.025)
w

1.959963984540054

In [13]:
def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """returns the symmetric (about the mean) bounds
    that contain the specified probability"""
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [14]:
u = normal_two_sided_bounds(0.95) 
u

(-1.959963984540054, 1.959963984540054)

In [15]:
# Unser Test wir die Anzahl von n Münzwürfen beinhalten,
# wobei wir in X zählen, wie oft Kopf fällt. Jeder Münzwurf ist ein Bernoulli-Experiment,
# daher ist X eine binomialverteilte Zufallsvariable B (n,p), 
# die folgenden Erwartungswert und Varianz hat:

In [16]:
def normal_approximation_to_binomial(n, p):
    """finds mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

In [17]:
# Experiment: Werfe Münze n = 1000 Mal, Annahme p = 0.5 (ausgewogene Münze)
# So sind die erwarten Werte wie folgt:

In [22]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
mu_0, sigma_0


(500.0, 15.811388300841896)

In [23]:
# Beispiel mit zweiseitigem Test:Betrachten wir einen Test, der H0 verwirft, falls X ausserhalb der folgenden Grenzen liegt:
# Signifikanzniveau = 5%

In [37]:
lo, hi  = normal_two_sided_bounds(0.95, 500.0, 15.8113883) 
lo, hi 

(469.010248386422, 530.9897516135779)

In [25]:
# Wir sind nun an der Sensitivität des Tests interessiert.
# = Wahrscheinlichkeit, keinen Fehler 2. Art zu begehen.
# Um die Sensitivität zu bestimmen,  braucht es die Alternativ-Hypothese H1
# Beispiel: H1: p = 0.55 (statt H0: p = 0.5) d.h. Kopf kommt etwas häufiger

In [26]:
# 95%- Grenze basierend auf der Annahme H0 p = 0.5

In [32]:
lo, hi = normal_two_sided_bounds(0.95, 500.0, 15.8113883)
lo, hi

(469.010248386422, 530.9897516135779)

In [28]:
# 95%- Grenze  basierend auf der Annahme H1 p = 0.55

In [33]:
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
mu_1, sigma_1

(550.0, 15.732132722552274)

In [30]:
# bei einem Fehler 2. Art liegt H1 vor, aber wir verwerfen J' irrtümlicherweise nicht
# was passiert, falls X immer noch im Intervall (469,530) liegt

In [34]:
type_2_probility = normal_probability_between (lo, hi, mu_1, sigma_1)
type_2_probility

0.11345221888145308

In [36]:
power_of_test = 1 - type_2_probility
power_of_test

0.8865477811185469

In [38]:
# Und nun ein Einseitiger Test
# Annahme: Münze neigt nicht dazu, übermässig auf Kopf zu fallen, d.h. p <= 0.5

In [39]:
hi = normal_upper_bound(0.95, mu_0, sigma_0 )
hi

526.0074193937779

In [40]:
# 526 ist kleiner als 531, weil wir mehr Wahrscheinlichkeit im oberen Bereich benötigen

In [41]:
type_2_probility = normal_probability_below (hi, mu_1, sigma_1)
type_2_probility

0.06362100214225695

In [42]:
power_of_test = 1 - type_2_probility
power_of_test

0.936378997857743

In [43]:
# Das 2. Beispiel ist ein aussagekräftigerer Test, 
# weil er nicht länger H0 verwirft, wenn X unterhalb von 469 liegt (was sehr unwahrscheinlich ist, wenn H1 zutrifft)
# Der 2. Test verwirft H0 schon früher, und zwar wenn X zwischen 526 und 531 liegt
# Dies ist besser, weil es einigermassen wahrscheinlich ist, wenn H1 zutrifft.


In [45]:
# Test mit Auswertung des P-Wertes

In [46]:
def two_sided_p_value(x, mu=0, sigma=1):
    if x >= mu:
        # if x is greater than the mean, the tail is above x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # if x is less than the mean, the tail is below x
        return 2 * normal_probability_below(x, mu, sigma)


In [47]:
u = normal_two_sided_bounds(0.95, mu_0, sigma_0 ) 
u

(469.0102483847719, 530.9897516152281)

In [50]:
probability_with_uncorrected_limit = normal_probability_between (530, 531, mu_0, sigma_0)
probability_with_uncorrected_limit

0.003927643541949966

In [52]:
probability_with_corrected_limit = normal_probability_between (529.5, 530.5, mu_0, sigma_0)
probability_with_corrected_limit

0.00417251711890243

In [53]:
# 0.00417 ist höher als 0.0039, daher Grenze 529.5 nehmen, d.h. dies ist eine bessere Schätzung für  530 Mal "Kopf"
# Kontinuitätskorrektur, en.wikipedia.org/Continuity_correction

In [54]:
p_value = two_sided_p_value(529.5, mu_0, sigma_0)
p_value

0.06207721579598835